<a href="https://colab.research.google.com/github/AjayKumarGogineni777/Heart-Sound-Segmentation/blob/master/LSTM_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import scipy.io as sio

mat_contents = sio.loadmat('/content/gdrive/My Drive/1D U Net/EWT/EWT_LS1_features.mat')
labels = sio.loadmat('/content/gdrive/My Drive/1D U Net/states_normal.mat')

In [0]:

### Stride 8
import numpy as np
import torch
elements = 56339
temp2 = torch.randn(elements,64)
cnt2 = 0
for i in range(792):
  a = labels['concat_states'][0][i].shape
  num = a[0]//8
  #print(num)
  for j in range(num):
    if j*8+64< a[0]:
      a2 = labels['concat_states'][0][i][j*8:(j*8+64)]
    
      a2 = torch.from_numpy(np.transpose(a2))
      temp2[cnt2] = a2
      cnt2 = cnt2+1
      
temp = torch.randn(elements, 4, 64)
cnt2 = 0
for i in range(792):
  a = mat_contents['LS1_features'][0][i].shape
  num = a[0]//8   
  
  for j in range(num):
    if j*8+64< a[0]:
      a2 = mat_contents['LS1_features'][0][i][j*8:(j*8+64)]
    
      a2 = torch.from_numpy(np.transpose(a2))
      temp[cnt2] = a2
      cnt2 = cnt2+1

In [0]:
## Satets/ Labels

import numpy as np
import torch
elements = 7104
    
import numpy as np
import torch
elements = 7104
temp2 = torch.randn(elements,64)
cnt2 = 0
for i in range(760):
  a = labels['concat_states'][0][i].shape
  num = a[0]//64
  #print(num)
  for j in range(num):
    a2 = labels['concat_states'][0][i][j*64:(j*64+64)]
    
    a2 = torch.from_numpy(np.transpose(a2))
    temp2[cnt2] = a2
    cnt2 = cnt2+1  

## Features
import numpy as np
import torch
elements = 7104
temp = torch.randn(elements,4,64)
cnt = 0
for i in range(760):
  a = mat_contents['LS1_features'][0][i].shape
  num = a[0]//64
  #print(num)
  for j in range(num):
    a2 = mat_contents['LS1_features'][0][i][j*64:(j*64+64)]
    a2 = torch.from_numpy(np.transpose(a2))
    #a2 = torch.from_numpy(np.array(a2))
    temp[cnt] = a2
    cnt = cnt+1
     
            
                

In [0]:
import torch
from torch import nn
# initializing the hidden state to 0
hidden=None

embeds_out = temp[0:64].transpose(1,2)
lstm = nn.LSTM(input_size=4, hidden_size=512, num_layers=3, batch_first=True)
lstm_out, h = lstm(embeds_out, hidden)
fc = nn.Linear(in_features=512, out_features=4)
fc_out = fc(lstm_out.contiguous().view(-1, 512))

print ('LSTM layer output shape', lstm_out.shape)

print ('FC layer output shape', fc_out.shape)
#print ('LSTM layer output ', lstm_out)

LSTM layer output shape torch.Size([64, 64, 512])
FC layer output shape torch.Size([4096, 4])


In [0]:
model = nn.Sequential(lstm, fc)

In [0]:
model.parameters()

<generator object Module.parameters at 0x7f1bdf0589e8>

In [0]:
out = fc_out.reshape(64,64,4)
out.shape

torch.Size([64, 64, 4])

In [0]:
import torch
from torch import nn
import torch.nn.functional as F
class mod1(nn.Module):
    '''(conv => BN => ReLU ) * 1'''
    def __init__(self):
        super(mod1, self).__init__()
        
        #self.lstm = nn.LSTM(input_size=4, hidden_size=256, num_layers=2, batch_first=True).cuda()
        self.lstm = nn.GRU(input_size=4, hidden_size=256, num_layers=2, batch_first=True).cuda()
        self.fc = nn.Linear(in_features=256, out_features=4).cuda()
    def forward(self, x, hidden):
        lstm_out, h = self.lstm(x, hidden)
        fc_out = self.fc(lstm_out.contiguous().view(-1, 256))
        out = fc_out.reshape(64,64,4)
        
        return out
        
model = mod1()        

In [0]:
#st = model.state_dict()
#mdl_path = '/content/gdrive/My Drive/1D U Net/models'
model = torch.load('/content/gdrive/My Drive/1D U Net/models/complete_adam_5_epochs')

In [0]:
mod1().parameters()

<generator object Module.parameters at 0x7f68d07bff68>

In [0]:
### ADAM optimiser

### training loop

import torch
from torch import nn
import torch.nn.functional as F
def softmax_mod(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()
def nll_indexing(soft,target):
  ## soft : 64,64,4
  ## target : 64,64,4 or 64, 64
  
  return -soft[range(target.shape[0]),range(target.shape[1]), target-1].mean()

def nll_final(soft,target):
  batch_var = torch.zeros([64, 64] , requires_grad=True)
  out = torch.zeros([1, 1] , requires_grad=True).cuda()
  for i in range(64):
    for j in range(64):
      ind = target[i][j]-1
      batch_var[i][j] = soft[i][j][ind]
      out = out + soft[i][j][ind]
  #out = batch_var.mean()
  return -1*(out/(64*64))


#soft = softmax_mod(out).transpose(1,2)
#soft.shape

lr = 5e-4
epochs = 6
n = 7391
bs = 64
loss_func = F.cross_entropy
loss_func2 = torch.nn.L1Loss
#def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

def accuracy_mod(out, yb): return (torch.argmax(out, dim=2)==yb-1).float().mean()
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

#model = LSTM()
#opt = torch.optim.Adam(model.parameters(), lr=lr)
import torch
from torch import nn
# initializing the hidden state to 0
hidden=None



#lstm = nn.LSTM(input_size=4, hidden_size=512, num_layers=3, batch_first=True).cuda()
#fc = nn.Linear(in_features=512, out_features=4).cuda()
#opt = torch.optim.Adam(list(lstm.parameters()) +
                             #list(fc.parameters()), lr=0.0001)##initially 1e-3
#lstm = torch.load('/content/gdrive/My Drive/1D U Net/models/lstm_adam_5_epochs')
#fc = torch.load('/content/gdrive/My Drive/1D U Net/models/fc_adam_5_epochs')


#model = mod1()



#opt = torch.optim.Adam(model.parameters(), lr = lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-2, amsgrad=False)
#opt = torch.optim.RMSprop(model.parameters(), lr = lr, alpha=0.99, eps=1e-08, weight_decay=1e-2, momentum=0, centered=False)

#opt = torch.optim.LBFGS(model.parameters(), lr = lr)
opt = torch.optim.Adam(model.parameters(), lr = lr)
#model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/models/small_complete_adam_230_epochs'))

model.load_state_dict(torch.load('/content/gdrive/My Drive/1D U Net/models/LS_EWT_GRU_34_epochs_stride_8'))

for epoch in range(epochs):
    for i in range(200): ### i = 1:100 for training
    #for i in range((n-1)//bs + 1):
#         set_trace()
        start_i = i*bs
        end_i = start_i+bs
        xb = temp[start_i:end_i].cuda()
        
        target = temp2[start_i:end_i].cuda()
        
        embeds_out = xb.transpose(1,2)
        #lstm_out, h = lstm(embeds_out, hidden)
        #fc_out = fc(lstm_out.contiguous().view(-1, 512))
        #out = fc_out.reshape(64,64,4)
        
        out = model(embeds_out, hidden)
        
        #loss = nll_final((softmax_mod(out.transpose(1, 2))),target.long()) ## Negative log likelihood of log softmax
        loss = nll_final((softmax_mod(out)),target.long()) ## Negative log likelihood of log softmax
        
        accuracy = accuracy_mod(softmax_mod(out),target.long())

        loss.backward()
     
        opt.step()
        opt.zero_grad()
    print('accuracy',accuracy)
    print(' > Epoch {:2d} loss: {:.3f}'.format((epoch+1), loss.item()))

accuracy tensor(0.6970, device='cuda:0')
 > Epoch  1 loss: 0.735
accuracy tensor(0.6785, device='cuda:0')
 > Epoch  2 loss: 0.775
accuracy tensor(0.6956, device='cuda:0')
 > Epoch  3 loss: 0.738
accuracy tensor(0.7112, device='cuda:0')
 > Epoch  4 loss: 0.719
accuracy tensor(0.7197, device='cuda:0')
 > Epoch  5 loss: 0.712
accuracy tensor(0.7310, device='cuda:0')
 > Epoch  6 loss: 0.653


In [0]:
st = model.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/models'
#torch.save(st, mdl_path+'/50_items_adam_24_epochs')
torch.save(st, mdl_path+'/LS_EWT_GRU_40_epochs_stride_8')

In [0]:
0.3178
0.299
0.283
0.278, 88.62 -174

0.275 88.40
0.273 88.53
0.279 88.26
0.270 88.75
0.272 88.67

In [0]:
#torch.cuda.empty_cache()

In [0]:
## Validation
#model.eval()
val_ids = list(range(80,109,1))
for i in val_ids: ### i = 1:100 for training, 100-114 for validation
    
    #for i in range((n-1)//bs + 1):
#         set_trace()
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i].cuda()
    #yb = y_f[start_i:end_i].cuda()
    target = temp2[start_i:end_i].cuda()
    embeds_out = xb.transpose(1,2)
    hidden=None
    out = model(embeds_out, hidden)
    
    loss = nll_final((softmax_mod(out)),target.long()) ## Negative log likelihood of log softmax
        
    accuracy = accuracy_mod(softmax_mod(out),target.long())

        
    
    print(' > Val {:2d} loss: {:.3f}'.format((i+1), loss.item()))
    print('accuracy',accuracy)

 > Val 81 loss: 0.480
accuracy tensor(0.8220, device='cuda:0')
 > Val 82 loss: 0.381
accuracy tensor(0.8552, device='cuda:0')
 > Val 83 loss: 0.511
accuracy tensor(0.7979, device='cuda:0')
 > Val 84 loss: 0.549
accuracy tensor(0.7842, device='cuda:0')
 > Val 85 loss: 0.392
accuracy tensor(0.8408, device='cuda:0')
 > Val 86 loss: 0.506
accuracy tensor(0.8020, device='cuda:0')
 > Val 87 loss: 0.634
accuracy tensor(0.7551, device='cuda:0')
 > Val 88 loss: 0.498
accuracy tensor(0.7991, device='cuda:0')
 > Val 89 loss: 0.604
accuracy tensor(0.7583, device='cuda:0')
 > Val 90 loss: 0.421
accuracy tensor(0.8320, device='cuda:0')
 > Val 91 loss: 0.455
accuracy tensor(0.8142, device='cuda:0')
 > Val 92 loss: 0.502
accuracy tensor(0.7937, device='cuda:0')
 > Val 93 loss: 0.860
accuracy tensor(0.6465, device='cuda:0')
 > Val 94 loss: 1.077
accuracy tensor(0.5452, device='cuda:0')
 > Val 95 loss: 0.417
accuracy tensor(0.8384, device='cuda:0')
 > Val 96 loss: 0.436
accuracy tensor(0.8359, device='c

In [0]:
63

In [0]:
sm_preds = []
bs = 64
for i in range(109):
    
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i].cuda()
    yb = y_f[start_i:end_i].cuda()
    target = temp2[start_i:end_i].cuda()
    embeds_out = xb.transpose(1,2)
    #lstm = nn.LSTM(input_size=4, hidden_size=512, num_layers=3, batch_first=True)
    #lstm_out, h = lstm(embeds_out, hidden)
    #fc = nn.Linear(in_features=512, out_features=4)
    #fc_out = fc(lstm_out.contiguous().view(-1, 512))
    out = model(embeds_out, hidden)  
    #out = model(xb)
    sm_preds_val = softmax_mod(out)
    sm_preds_fn = torch.argmax(sm_preds_val, dim=2)+1
    sm_preds.append(sm_preds_fn)

In [0]:
out.shape

torch.Size([64, 64, 4])

In [0]:
tr = []
pred = []
for i in range(80,108):
    start_i = i*bs
    end_i = start_i+bs
    xb = temp[start_i:end_i]
    yb = y_f[start_i:end_i]
    target = temp2[start_i:end_i]
    s1_true = np.where(temp2[start_i:end_i][0] == 1)
    temp234 = sm_preds[i].cpu()
    s1_pred = np.where(temp234[0] == 1)
    tr.append(s1_true)
    pred.append(s1_pred)

In [0]:
s1_true

(array([18, 19, 20, 21, 22, 53, 54, 55, 56, 57]),)

In [0]:
i = 105
start_i = i*bs
end_i = start_i+bs
np.where(temp2[start_i:end_i][0]==3)

(array([44, 45, 46, 47, 48]),)

In [0]:
sm_preds[0]

tensor([[4, 4, 4,  ..., 2, 2, 2],
        [4, 4, 1,  ..., 4, 4, 4],
        [4, 4, 4,  ..., 2, 2, 2],
        ...,
        [4, 4, 4,  ..., 4, 1, 1],
        [3, 1, 1,  ..., 2, 2, 2],
        [4, 4, 3,  ..., 4, 4, 4]], device='cuda:0')

In [0]:
s1_pred

(array([], dtype=int64),)

In [0]:
for i in range(29):
    print(i)
    print(tr[i])
    print(pred[i])

0
(array([23, 24, 25, 26, 27, 28, 29]),)
(array([20, 21, 22, 23, 24, 25, 26, 27]),)
1
(array([19, 20, 21, 22, 23, 24, 25, 57, 58, 59, 60, 61, 62, 63]),)
(array([ 0,  1,  2,  3, 21, 22, 23, 24, 25, 26, 57, 58, 59, 60, 61, 62, 63]),)
2
(array([ 0,  1,  2,  3,  4,  5,  6, 37, 38, 39, 40, 41, 42, 43]),)
(array([37, 38, 39, 40, 41, 42, 43, 44]),)
3
(array([ 6,  7,  8,  9, 10, 11, 12]),)
(array([ 7,  8,  9, 10, 11, 12, 13]),)
4
(array([ 0,  1, 54, 55, 56, 57, 58, 59, 60]),)
(array([ 0, 55, 56, 57, 58, 59, 60]),)
5
(array([36, 37, 38, 39, 40, 41, 42]),)
(array([ 2, 34, 35, 36, 37, 38, 39, 40, 41]),)
6
(array([13, 14, 15, 16, 17, 18, 19, 58, 59, 60, 61, 62, 63]),)
(array([16, 17, 18, 19, 20, 21, 60, 61, 62, 63]),)
7
(array([18, 19, 20, 21, 22, 23, 24, 58, 59, 60, 61, 62, 63]),)
(array([18, 19, 20, 21, 22, 23, 24, 59, 60, 61, 62, 63]),)
8
(array([ 6,  7,  8,  9, 10, 11, 12, 44, 45, 46, 47, 48, 49, 50]),)
(array([ 7,  8,  9, 10, 11, 12, 13, 45, 46, 47, 48, 49, 50, 51]),)
9
(array([ 0, 31, 32, 33

IndexError: ignored

In [0]:
st = lstm.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/models'
torch.save(st, mdl_path+'/lstm_adam_5_epochs')

In [0]:
st = fc.state_dict()
mdl_path = '/content/gdrive/My Drive/1D U Net/models'
torch.save(st, mdl_path+'/fc_adam_5_epochs')

In [0]:
lstm = torch.load('/content/gdrive/My Drive/1D U Net/models/lstm_adam_5_epochs')
fc = torch.load('/content/gdrive/My Drive/1D U Net/models/fc_adam_5_epochs')

In [0]:
lstm

In [0]:
start_i = 0
end_i = 1
bs = 64
xb = temp[start_i:end_i]
xb.shape